In [64]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time, os
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
#link to googles Shang-Chi page will still need to use selenium to get to reviews page and load all of the reviews

base_link = 'https://www.google.com/search?q=shang-chi+reviews&rlz=1C5CHFA_enUS949US951&oq=shang-chi+reviews&aqs=chrome.0.69i59j35i39j46i131i433i512j0i131i433i512l2j69i60l3.1797j0j7&sourceid=chrome&ie=UTF-8'

In [4]:
page = requests.get(base_link).text
soup = BeautifulSoup(page, 'html5lib')

In [5]:
driver =webdriver.Chrome(chromedriver)

In [6]:
driver.get(base_link)

In [7]:
driver.page_source[:1000]

'<html itemscope="" itemtype="http://schema.org/SearchResultsPage" lang="en"><head><meta charset="UTF-8"><meta content="origin" name="referrer"><meta content="Anb2GUnhMjfTIX0D2a4a6NPAqPI5GaxxRAiF81XTjHJ2qK4E3Hw3VFM4eaJBgRzz45CNPt624audv+wHOJwfAAEAAABieyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IlRydXN0VG9rZW5zIiwiZXhwaXJ5IjoxNjI2MjIwNzk5LCJpc1N1YmRvbWFpbiI6dHJ1ZX0=" http-equiv="origin-trial"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>shang-chi reviews - Google Search</title><script nonce="Q5gfhQEsSGHTNGon+UxAmA==">(function(){window.google={kEI:\'apA-YfmnBJLm9APJhKXQBw\',kEXPI:\'31\',kBL:\'myGb\'};google.sn=\'web\';google.kHL=\'en\';})();(function(){\nvar f=this||self;var h,k=[];function l(a){for(var b;a&&(!a.getAttribute||!(b=a.getAttribute("eid")));)a=a.parentNode;return b||h}function m(a){for(var b=null;a&&(!a.getAttribute||!(b=a.getAttribute("leid")));)a=a.parentNode;return b}\nfunction n(a,b,c,d,g){var e="";

In [8]:



review_button = driver.find_element_by_xpath("//*[contains(text(), 'More audience reviews')]")
review_button.click()



In [9]:
soup = BeautifulSoup(driver.page_source)


In [10]:
driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

In [61]:
#get all reviews from google and put them in a list of docs

soup.find_all('pre jsname'=='pkrv7',class_="T7nuU")
soup = BeautifulSoup(driver.page_source)

all_reviews = soup.find_all('pre jsname'=='pkrv7',jsname="QUIPvd")

review_list=[]
for item in all_reviews:
    review_list.append(item.text)
review_list

#'star-rating class'=='ePCv9'

["Shang-chi review : spectacular visual wonder\nMy rating : 4/5\nCast & performances : \nThe film has Simu Liu in top form. He is your ordinary guy but extraordinary powers and actor blends into the character well - with not a single false note. Giving him good company are actresses Awkwafina who is effortless in whatever she does and Meng'er Zhang who also performs some mind-numbing stunts. \nThe star of the film of course is veteran actor Tony Leung- a father who wants to bring back his family together but also is aware of his extraordinary powers. It's his love for his dead wife that makes him tough and cold towards his children. Years back Leung had charmed the world with his subtle style in Wong Kar Wai's 'In The Mood For Love'. Decades later, Leung has aged like fine wine and it's a delight to watch him play the antagonist with such style. \nBen Kingsley and Michelle Yeoh too make their presence felt in the limited screen time they have. \n\nTechnical values : \nShang-Chi scores 

In [74]:
review_dict={}
for idx,review in enumerate(review_list):
    review_dict[idx]=review

In [171]:
doc_df = pd.DataFrame.from_dict(review_dict, orient='index')
doc_df.rename(columns={0:'review'},inplace=True)
doc_df

,review
0,Shang-chi review : spectacular visual wonder\n...
1,The much-awaited Shangi Chi and the Legend of ...
2,"Like many, I was concerned that this new Marve..."
3,"No movie is without flaws, but this was a brea..."
4,Hands down one of the best MCU Movies. Every a...
...,...
2201,Marvel is Back 👍😘😍 😎😎😎😎😎😎😎😎😎😎😎😎😎😎
2202,boring...........................................
2203,⭐⭐⭐⭐⭐⭐\n \n\n \n \n .............................
2204,thats what she said..............................


In [172]:
# Text preprocessing steps - remove numbers, captial letters and punctuation

import re
import string

alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

doc_df['review'] = doc_df.review.map(alphanumeric).map(punc_lower)
doc_df


,review
0,shang chi review spectacular visual wonder\n...
1,the much awaited shangi chi and the legend of ...
2,like many i was concerned that this new marve...
3,no movie is without flaws but this was a brea...
4,hands down one of the best mcu movies every a...
...,...
2201,marvel is back 👍😘😍 😎😎😎😎😎😎😎😎😎😎😎😎😎😎
2202,boring ...
2203,⭐⭐⭐⭐⭐⭐\n \n\n \n \n ...
2204,thats what she said ...


In [173]:
#create function to remove the '/n' and remove extra spaces

def clear_n(string):
    str(string)
    string=string.replace('\n',' ')
    return string

def strip_string(string):
    string.strip
        
clear_n('shang chi review spectacular visual wonder\n')

print(strip_string('shang chi review spectacular visual wonder    '))

None


In [174]:
doc_df['review']=doc_df['review'].apply(clear_n)



doc_df
doc_df['review']=doc_df['review'].apply(str.strip)

doc_df

,review
0,shang chi review spectacular visual wonder m...
1,the much awaited shangi chi and the legend of ...
2,like many i was concerned that this new marve...
3,no movie is without flaws but this was a brea...
4,hands down one of the best mcu movies every a...
...,...
2201,marvel is back 👍😘😍 😎😎😎😎😎😎😎😎😎😎😎😎😎😎
2202,boring
2203,⭐⭐⭐⭐⭐⭐ ...
2204,thats what she said


In [175]:
doc_df.to_csv('shang_chi_reviews.csv')

In [176]:
driver.quit()    